In [ ]:
import torch as tch
from torch import nn
import torchaudio as ta
import torchaudio.transforms as tatx
import torchaudio.functional as tafn
from pathlib import Path
import plotly.express as px
import pandas as pd
import numpy as np
from IPython.display import Audio

from train_tools import find_lr, check_with_patient, UpdatingPlotlyLines, fit_step, evaluate
from torch.optim import Adam
from torch.nn import functional as nnfn

device = tch.device("cpu")

%load_ext autoreload
%autoreload 2 

In [ ]:
import prep_sample as pl

In [ ]:
we_df = pd.read_csv('temp/we_df.csv')
we_seg_df = pd.read_csv('temp/wee_seg_16000.csv', index_col=0)

idx = np.random.choice(len(we_df), len(we_df), replace=False)
we_df_shuffled = we_df.iloc[idx]
we_seg_df_shuffled = we_seg_df.iloc[idx]

pos_val_size = 500
pos_train_ds = pl.CommonVoiceDataset('dataset/Common Voice Corpus 1/en/', 16000)
pos_train_ds.use_df(we_df_shuffled[:-pos_val_size])
we_seg_df_shuffled_train = we_seg_df_shuffled[:-pos_val_size]

pos_val_ds = pl.CommonVoiceDataset('dataset/Common Voice Corpus 1/en/', 16000)
pos_val_ds.use_df(we_df_shuffled[-pos_val_size:])
we_seg_df_shuffled_val = we_seg_df_shuffled[-pos_val_size:]

In [ ]:
pos_train_ds[0]

In [ ]:
Audio(pl.get_middle_segment(pos_train_ds, we_seg_df_shuffled_train, 1, 400), rate=16000)

In [ ]:
pl.show_spectrum(pl.get_middle_segment(pos_train_ds, we_seg_df_shuffled_train, 1, 400)[0], 16000)

In [ ]:
x = pl.get_segment(pos_train_ds, we_seg_df_shuffled_train, 1)
pl.show_spectrum(x[0], 16000)

In [ ]:
def repeat_last_frame(x, idx=-1, n_times=100): 

    sxx = tatx.Spectrogram(power=2)(x)
    x_ = tatx.GriffinLim()(sxx[..., [idx]*n_times])[0]
    return x_

x = pl.get_segment(pos_train_ds, we_seg_df_shuffled_train, 1)*1e12
Audio(repeat_last_frame(x, -3), rate=16000)

In [ ]:
Audio(x, rate=16000)

In [ ]:
pl.show_spectrum(x[0], 16000)

In [ ]:
px.line(x[0])

In [ ]:
x = pos_train_ds[0][0][0]

In [ ]:
px.line(tafn.detect_pitch_frequency(x/100, 16000, win_length=30))

In [ ]:
Audio(x, rate=16000)

In [ ]:
tafn.detect_pitch_frequency?